# 09. Lab04-1 Multi-Variable Linear regression 구현

- 지금까지는 1개의 Variable을 대상으로 Linear regresion을 수행
- 사실상 toy project
- 현실의 문제는 여러개의 variable로 부터 결과를 도출
- 현실의 문제를 해결하는 초석 마련

## Hypothesis using matrix

| $x_1$(quiz 1) | $x_2$(quiz 2) | $x_3$(midterm 1)|Y(final)|
|---|---|---|---|
|73|80|75|152|
|93|88|93|185|
|89|91|90|180|
|96|98|100|196|
|73|66|70|142|

- $ H(x_1, x_2, x_3) = x_1w_1 + x_2w_2 + x_3w_3 $ (bias 생략)

- 위 데이터는 variable이 3개임
- Variable이 많으면 많을수록 문제 해결에 도움이 됨
- Variable이 많으면 많을수록 예측률이 좋아짐



In [19]:
import tensorflow as tf

x1_data = [73., 93., 89., 96., 73.]
x2_data = [80., 88., 91., 98., 66.]
x3_data = [75., 93., 90., 100., 70.]
y_data = [152., 185., 180., 196., 142.]

X1 = tf.placeholder(tf.float32)
X2 = tf.placeholder(tf.float32)
X3 = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([1]), name="weight1")
W2 = tf.Variable(tf.random_normal([1]), name='weight2')
W3 = tf.Variable(tf.random_normal([1]), name='weight3')
b = tf.Variable(tf.random_normal([1]), name='bias')

hypothesis = X1*W1 + X2*W2 + X3*W3 + b
cost = tf.reduce_mean(tf.square(hypothesis - Y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(100001):
    h_val, c_val, _ = sess.run([hypothesis, cost, train], 
                               feed_dict={X1:x1_data, 
                                          X2:x2_data, 
                                          X3:x3_data, 
                                          Y:y_data})
    if step%10000 == 0:
        print(step, h_val, c_val) #h_val은 hypothesis가 예측한 각 인스턴스 별 데이터

0 [ 118.12126923  147.77737427  142.64335632  154.09455872  115.34895325] 1279.03
10000 [ 151.71638489  184.56575012  180.74475098  195.55761719  142.53894043] 0.261965
20000 [ 151.78500366  184.50483704  180.74784851  195.69717407  142.34359741] 0.21209
30000 [ 151.74455261  184.52171326  180.72134399  195.78578186  142.2752533 ] 0.187201
40000 [ 151.70677185  184.53909302  180.69862366  195.85397339  142.2270813 ] 0.171877
50000 [ 151.67652893  184.55316162  180.68057251  195.90734863  142.18991089] 0.162425
60000 [ 151.65290833  184.5640564   180.66636658  195.94923401  142.16065979] 0.156591
70000 [ 151.63441467  184.57264709  180.6552124   195.98216248  142.13778687] 0.152978
80000 [ 151.61972046  184.57948303  180.64633179  196.00784302  142.12008667] 0.150735
90000 [ 151.60845947  184.58467102  180.63938904  196.0282135   142.105896  ] 0.149326
100000 [ 151.5995636   184.58877563  180.63388062  196.04414368  142.09489441] 0.148443


![](./images/08/log.jpg)

- 이 접근법의 문제점
  - Variable이 늘어나면 Hypothesis를 코드로 수정해야 함
    - 수정 범위
      - Hypothesis의 공식이 변경됨
      - sess.run의 feed_diect를 변경해야 함
      - Placeholder를 추가해야 함
      - Variable을 추가해야 함
      - 변수 별로 data를 추가해야 함
- 변경 포인트가 많음

![](./images/08/nogood.jpg)

## Matrix를 적용

$$
\left[ 
\begin{array}{cccc}
x_1 & x_2 & x_3 \\
\end{array} 
\right] 
\left[ 
\begin{array}{cccc}
w_1 \\
w_2 \\
w_3 \\
\end{array} 
\right]
= 
\left[ 
\begin{array}{cccc}
x_1w_1 + x_2w_2 + x_3w_3 \\
\end{array} 
\right]
$$

$$
H(X) = XW
$$

- 행렬을 적용할 경우 Hypethesis 공식이 단순화됨
- feature가 추가됨에 따라 Hypothesis를 변경할 필요가 없음

### matrix를 적용할 경우 변경 코드

```python
x_data = [ 
           [73, 80, 75], [93, 88, 93],
           [89, 91, 90], [96, 98, 100],
           [73, 66, 70]
         ]
y_data = [[152], [185], [180], [196], [142]]
X = tf.placeholder(tf.float32, shape=[None, 3]
y = tf.placeholder(tf.float32, shape=[None, 1]

W = tf.Variable(tf.random_normal([3, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

hypothesis = tf.matmul(X, x) + b  
```

### feature 변경시 코드 변경 대상

![](./images/08/afterMatrix.jpg)

### 전체 구현 코드

In [27]:
import tensorflow as tf

x_data = [ 
           [73, 80, 75], [93, 88, 93],
           [89, 91, 90], [96, 98, 100],
           [73, 66, 70]
         ]
y_data = [[152], [185], [180], [196], [142]]
X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([3, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

hypothesis = tf.matmul(X,W ) + b  

In [26]:
cost = tf.reduce_mean(tf.square(hypothesis - Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(10001):
    cost_val, hy_val, _ = sess.run([cost, hypothesis, train], 
                                   feed_dict={X:x_data, Y:y_data})
    if step % 2500 == 0:
         print(step, cost_val, hy_val)



0 95559.7 [[-122.64464569]
 [-145.31958008]
 [-144.25973511]
 [-155.90328979]
 [-111.53009796]]
2500 1.19978 [[ 151.17993164]
 [ 184.6807251 ]
 [ 180.44485474]
 [ 197.62384033]
 [ 140.45413208]]
5000 0.999445 [[ 150.96800232]
 [ 184.83691406]
 [ 180.39372253]
 [ 197.47923279]
 [ 140.75001526]]
7500 0.885621 [[ 150.88172913]
 [ 184.90629578]
 [ 180.38026428]
 [ 197.36824036]
 [ 140.92663574]]
10000 0.801212 [[ 150.85804749]
 [ 184.93218994]
 [ 180.38522339]
 [ 197.27656555]
 [ 141.04115295]]


## last update

In [29]:
import datetime
print(datetime.datetime.now())

2017-05-31 05:57:30.962098
